# Problem 54

### Poker hands

In the card game poker, a hand consists of five cards and are ranked, from lowest to highest, in the following way:

**High Card**: Highest value card

**One Pair**: Two cards of the same value

**Two Pairs**: Two different pairs

**Three of a Kind**: Three cards of the same value

**Straight**: All cards are consecutive values

**Flush**: All cards of the same suit

**Full House**: Three of a kind and a pair

**Four of a Kind**: Four cards of the same value

**Straight Flush**: All cards are consecutive values of same suit

**Royal Flush**: Ten, Jack, Queen, King, Ace, in same suit

The cards are valued in the order:
2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King, Ace.

The file, poker.txt, contains one-thousand random hands dealt to two players. Each line of the file contains ten cards (separated by a single space): the first five are Player 1's cards and the last five are Player 2's cards. You can assume that all hands are valid (no invalid characters or repeated cards), each player's hand is in no specific order, and in each hand there is a clear winner.

How many hands does Player 1 win?

##### Solution

In [1]:
import pandas as pd

In [2]:
hands = pd.read_table("Files/Problem 54/poker.txt",header=None,delimiter=' ')

In [3]:
player1 = hands[[0,1,2,3,4]]
player2 = hands[[5,6,7,8,9]].rename({5:0, 6:1, 7:2, 8:3, 9:4},axis=1)

In [4]:
cards = {'2': 2,
         '3': 3,
         '4': 4,
         '5': 5,
         '6': 6,
         '7': 7,
         '8': 8,
         '9': 9,
         'T': 10,
         'J': 11,
         'Q': 12,
         'K': 13,
         'A': 14}

In [5]:
poker_hands = {'High card': 1,
               'Pair': 2,
               'Two pairs': 3,
               'Three of a kind': 4,
               'Straight': 5,
               'Flush': 6,
               'Full house': 7,
               'Four of a kind': 8,
               'Straight flush': 9
              }

In [6]:
def check_flush(hand):
    old_suit = hand[0][1]
    values = []
    
    for card in hand:
        new_suit = card[1]
        if new_suit != old_suit:
            return False,None
        values.append(cards[card[0]])
    
    values.sort(reverse=True)
    return True,values

In [7]:
test_false = pd.Series(['AC','9H','TC','2C','JH'])
test_true = pd.Series(['AH','9H','TH','2H','JH'])

print(check_flush(test_false))
print(check_flush(test_true))

(False, None)
(True, [14, 11, 10, 9, 2])


In [8]:
def check_straight(hand):
    values = []
    for card in hand:
        values.append(cards[card[0]])
    
    values.sort(reverse=True) 
    if values == [14,5,4,3,2]:
        return True,[5,4,3,2,1]
    
    for i in range(1,len(values)):
        if values[i] != values[i-1] - 1:
            return False,None
    
    return True,values

In [9]:
test_false = pd.Series(['3C','5H','AD','AC','6C'])
test_true = pd.Series(['QH','9H','TC','KD','JH'])

print(check_straight(test_false))
print(check_straight(test_true))

(False, None)
(True, [13, 12, 11, 10, 9])


In [24]:
def number_combinations(hand):
    values = []
    for card in hand:
        values.append(cards[card[0]])
    
    matches = pd.Series(values).value_counts()
    sequence = pd.DataFrame(data=[matches.index,matches]).transpose().sort_values(by=[1,0],ascending=False)[0].values
    
    if len(matches) == 2:
        if max(matches) == 4:
            return 'Four of a kind',sequence
        return 'Full house', sequence
    if len(matches) == 3:
        if max(matches) == 3:
            return 'Three of a kind',sequence
        return 'Two pairs', sequence
    if len(matches) == 4:
        return 'Pair',sequence
    return 'High card',sequence

In [25]:
four_of_a_kind = pd.Series(['3C','3H','3D','3S','5H'])
full_house = pd.Series(['QH','9C','9H','9D','QD'])
three_of_a_kind = pd.Series(['3C','3H','3S','AC','5H'])
two_pairs = pd.Series(['QH','QS','TH','TC','JH'])
pair = pd.Series(['3C','6H','AC','AC','5H'])
high_card = pd.Series(['QH','9H','TH','KH','JH'])

print(number_combinations(four_of_a_kind))
print(number_combinations(full_house))
print(number_combinations(three_of_a_kind))
print(number_combinations(two_pairs))
print(number_combinations(pair))
print(number_combinations(high_card))

('Four of a kind', array([3, 5], dtype=int64))
('Full house', array([ 9, 12], dtype=int64))
('Three of a kind', array([ 3, 14,  5], dtype=int64))
('Two pairs', array([12, 10, 11], dtype=int64))
('Pair', array([14,  6,  5,  3], dtype=int64))
('High card', array([13, 12, 11, 10,  9], dtype=int64))


In [26]:
def compare_vectors(v1,v2):
    """
    Compares two vectors of the same size based on their earliest different values and returns [1,0] if the first is greater,
    [0,1] if the second is greater and [0,0] if they are the same
    
    Input:
    - v1,v2: vectors of integers
    
    Return: [int,int]
    """
    for i in range(len(v1)):
        if v1[i] > v2[i]:
            return [1,0]
        if v2[i] > v1[i]:
            return [0,1]
    
    return [0,0]

In [30]:
def compare_hands(hand1,hand2):
    p1_flush,p1_values = check_flush(hand1)
    p2_flush,p2_values = check_flush(hand2)
    p1_straight,p1_straight_cards = check_straight(hand1)
    p2_straight,p2_straight_cards = check_straight(hand2)
    
    if p1_flush or p1_straight:
        if p1_flush and p1_straight:
            p1_result = poker_hands['Straight flush']
            p1_cards = p1_values
        elif p1_flush:
            p1_result = poker_hands['Flush']
            p1_cards = p1_values
        else:
            p1_result = poker_hands['Straight']
            p1_cards = p1_straight_cards
    else:
        result,p1_cards = number_combinations(hand1)
        p1_result = poker_hands[result]
    
    if p2_flush or p2_straight:
        if p2_flush and p2_straight:
            p2_result = poker_hands['Straight flush']
            p2_cards = p1_values
        elif p2_flush:
            p2_result = poker_hands['Flush']
            p2_cards = p1_values
        else:
            p2_result = poker_hands['Straight']
            p2_cards = p1_straight_cards
    else:
        result,p2_cards = number_combinations(hand2)
        p2_result = poker_hands[result]
    
    if p1_result != p2_result:
        if p1_result > p2_result:
            return [1,0]
        else:
            return [0,1]
    return compare_vectors(p1_cards,p2_cards)

In [31]:
p1 = 0
p2 = 0

for i in range(len(hands)):
    result = compare_hands(player1.iloc[i],player2.iloc[i])
    p1 += result[0]
    p2 += result[1]

print(p1,p2)

376 624
